Clean the provided sales_data.csv datsetn analyze sales performance across regions, products, customer segments, and time. Produce at least 4 visualizations, answer the business questions below and submit a cleaned CSV, a jupyter notebook with code + comments and a one page summary with top 5 insights and recommendations.

Business questions to answer

Which region generated the highest total revenue?

Which products are top revenue contributors?

What is the monthly sales trend -- any seasonally?

Does discount level affect revenue/quality sold?

Which payment and shipping methods are most used nd most profitable?


#### Quick Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_columns = 60
pd.options.display.max_rows = 20

plt.rcParams['figure.figsize'] = (10,6)

#### Load Data

In [5]:
df = pd.read_csv('sales_data.csv')
print('Shape of df', df.shape)
display(df.head())

Shape of df (2580, 13)


,Order_ID,Date,Customer_ID,Product,Category,Quantity,Price,Discount,Region,Payment_Method,Customer_Segment,Shipping_Method,Revenue
0,1,2022-04-13,CUST1759,Smartwatch,Electronics,1.0,745.0,0.15,East,Online Wallet,Consumer,Same Day,633.25
1,2,2023-03-12,CUST1776,Camera,Accessories,17.0,1066.0,0.00,West,Credit Card,Corporate,First Class,18122.00
2,3,2022-09-28,CUST1726,Headphones,Accessories,4.0,379.0,0.15,South,Debit Card,Home Office,Same Day,1288.60
3,4,2022-04-17,CUST1200,Tablet,Office Supplies,5.0,709.0,0.10,East,Cash,Small Business,Same Day,3190.50
4,5,2022-03-13,CUST1079,Tablet,Accessories,7.0,1690.0,0.05,North,Online Wallet,Consumer,First Class,11238.50


#### Quick structure summary (EDA Start)

In [7]:
df.info() #dtype and non-null values count

display(df.describe(include = 'all').transpose())

print('Missing values per column\n',df.isnull().sum())

print('Number of duplicate rows', df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2580 entries, 0 to 2579
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          2580 non-null   int64  
 1   Date              2580 non-null   object 
 2   Customer_ID       2580 non-null   object 
 3   Product           2528 non-null   object 
 4   Category          2580 non-null   object 
 5   Quantity          2528 non-null   float64
 6   Price             2527 non-null   float64
 7   Discount          2580 non-null   float64
 8   Region            2580 non-null   object 
 9   Payment_Method    2527 non-null   object 
 10  Customer_Segment  2580 non-null   object 
 11  Shipping_Method   2580 non-null   object 
 12  Revenue           2475 non-null   float64
dtypes: float64(4), int64(1), object(8)
memory usage: 262.2+ KB


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Order_ID,2580.0,NaN,NaN,NaN,1249.60155,720.858175,1.0,626.75,1251.5,1872.25,2500.0
Date,2580,695,2022-04-09,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Customer_ID,2580,750,CUST1154,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Product,2528,7,Printer,389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Category,2580,3,Accessories,908,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,2528.0,NaN,NaN,NaN,10.085047,5.531316,1.0,5.0,10.0,15.0,19.0
Price,2527.0,NaN,NaN,NaN,1032.014246,552.643569,50.0,563.5,1044.0,1512.0,1999.0
Discount,2580.0,NaN,NaN,NaN,0.100136,0.070731,0.0,0.05,0.1,0.15,0.2
Region,2580,4,East,663,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Payment_Method,2527,5,Debit Card,525,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Missing values per column
 Order_ID              0
Date                  0
Customer_ID           0
Product              52
Category              0
Quantity             52
Price                53
Discount              0
Region                0
Payment_Method       53
Customer_Segment      0
Shipping_Method       0
Revenue             105
dtype: int64
Number of duplicate rows 80


Why

info:- reveals wrong types(eg.date as object).

isnull().sum():- tells where cleaning is needed.

duplicated():- show the duplicates

#### convert date to datetime and sort by date

In [10]:
df['Date'] = pd.to_datetime(df['Date'], errors= 'coerce') #bad parse -> NaT

print('Date parsing NaT count: ', df['Date'].isna().head())

# Optional inspects bad rows

display(df[df['Date'].isna()].head())

# For analysis drop rows without a valid date

df = df.dropna(subset= ['Date']).copy()
df = df.sort_values('Date').reset_index(drop= True)

Date parsing NaT count:  0    False
1    False
2    False
3    False
4    False
Name: Date, dtype: bool


,Order_ID,Date,Customer_ID,Product,Category,Quantity,Price,Discount,Region,Payment_Method,Customer_Segment,Shipping_Method,Revenue


#### Why

Time series operations require true datetime

errors = 'coerce' helps catch malformed dates

#### Ensure numeric types for Quantity, Price, Discount

In [14]:
df['Quantity'] = pd.to_numeric(df['Quantity'],errors='coerce')

df['Price'] = pd.to_numeric(df['Price'],errors='coerce')

df['Discount'] = pd.to_numeric(df['Discount'],errors='coerce')

print(df[['Quantity', 'Price', 'Discount']].dtypes)

print('Nulls after coercion:\n',df[['Quantity', 'Price', 'Discount']].isnull().sum())

Quantity    float64
Price       float64
Discount    float64
dtype: object
Nulls after coercion:
 Quantity    52
Price       53
Discount     0
dtype: int64


#### Handling Missing values

General Strategy:

1. Drop rows missing essential categorical info like product

2. For Quantity: fill with median

3. For price: fill with median price

4. For payment_method = fill unknown

In [ ]:
#1. Drop rows with missing product (can't assign revenue)

df = df.dropna(subset = ['Product'])

#2. Fill Quantity with median

qty = int(df['Quantity'].median())

df['Quantity'].fillna(qty, inplace= True)

#3. Fill price with Product Level median

df['Price'] = df['Price'].fillna(df.groupby('Product')['Price'].transform('median'))

df['Price'].fillna(df['Price'].median(), inplace= True)

#4. Fill Payment_method = unknown

df['Payment_Method'].fillna('Unknown', inplace= True)

# Check

print('Missing after inputation', df.isnull().sum())




Missing after inputation Order_ID              0
Date                  0
Customer_ID           0
Product               0
Category              0
Quantity              0
Price                 0
Discount              0
Region                0
Payment_Method        0
Customer_Segment      0
Shipping_Method       0
Revenue             103
dtype: int64


C:\Users\sarth\AppData\Local\Temp\ipykernel_28064\2327077593.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Quantity'].fillna(qty, inplace= True)
C:\Users\sarth\AppData\Local\Temp\ipykernel_28064\2327077593.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [ ]:
#### Remove duplicate